In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lxml import html
import re
import json
import time

In [2]:
#структура сайта
main_url = "http://marspb.ru"
types = ['tram', 'trolleybus', 'bus', 'marshrutka']

In [3]:
#scrapper
def getRoutsHrefFromPage(page_href):
    url = main_url + page_href
    
    r = requests.get(url)
    text = r.text
    soup = BeautifulSoup(text, "lxml")
    
    rout_items = soup.find_all('div', {'class': 'route-item'})
    hrefs = list(map(lambda rout: rout.find('a').get('href'), rout_items))
    numbers = list(map(lambda rout: rout.find('a').text, rout_items))
    
    routs = []
    for i in range(0, len(rout_items)):
        rout = {'href':hrefs[i], 'number': numbers[i]}
        routs.append(rout)
        
    return routs

def getRoutsHrefs():
    routs = []
    
    for mar_type in types:
        href = "/" + mar_type + "/"
         
        routs_part = getRoutsHrefFromPage(href)
        wtf = list(map(lambda rout: rout.update({'type':mar_type}), routs_part))
        
        routs.extend(routs_part)
        
        #Необходимо обходить страницы
        url = main_url + href
        r = requests.get(url)
        text = r.text
        soup = BeautifulSoup(text, "lxml")
        
        page_wrapper = soup.find('div', {'class': 'pager-wrapper'})
        if page_wrapper:
            pages_href = page_wrapper.find_all('a')
            pages = list(map(lambda x : x.get('href'), pages_href))
            
            for page in pages:
                routs_part = getRoutsHrefFromPage(page)
                wtf = list(map(lambda rout: rout.update({'type':mar_type}), routs_part))
                routs.extend(routs_part)
    
    return(routs)

def getRout(rout):
    url = main_url + rout['href']
    r = requests.get(url)
    text = r.text
    soup = BeautifulSoup(text, 'lxml')
    
    rout_text = soup.find_all('script')[-1].text
    rout_text = re.sub("^\s+|\n|\r|\t|\s+$", '', rout_text)
    rout_text = re.sub('.*Route: ', '', rout_text)
    rout_text = re.sub(',MapIconsDirectory.*', '', rout_text)
    
    rout_dict = json.loads(rout_text)
    rout_pf = rout_dict['pf'] if 'pf' in rout_dict else {}
    rout_pb = rout_dict['pb'] if 'pb' in rout_dict else {}
    
    rout.update({'pf':rout_pf, 'pb': rout_pb})
    
    return rout
    

def getAllRouts():    
    routs = getRoutsHrefs()
    for rout in routs:
        getRout(rout)
        #time.sleep(1)
        
        
    return routs

In [4]:
routs = getAllRouts()

In [5]:
rout_pf = list(map(lambda r: {"href":r["href"], "number": r["number"], "direction": "f", "type": r['type'], "rout": r['pf']}, routs))
rout_pb = list(map(lambda r: {"href":r["href"], "number": r["number"], "direction": "b", "type": r['type'], "rout": r['pb']}, routs))

In [6]:
routs = []
routs.extend(rout_pf)
routs.extend(rout_pb)

In [7]:
#convert to geojson
geojson = {
    "type": "FeatureCollection",
    
    "features": [
    {
        "type": "Feature",
        "properties": {
            "href" : r["href"],
            "number" : r["number"],
            "direction": r["direction"],
            "type": r["type"]
        },
        "geometry" : {
            "type": "LineString",
            "coordinates": [[rc["lng"], rc["lat"]] for rc in r['rout']],
            },
        
     } for r in routs]
}

In [10]:
output = open("public_transport_routs_3.geojson", 'w')
json.dump(geojson, output)

In [29]:
#stops
stops_geojson = {"type": "FeatureCollection", "features" : []}
for r in routs:
    for rc in r['rout']:
        if rc['s'] == True:
            feature =  {
                        "type": "Feature",
                        "properties": {
                            "rout" : r["href"],
                            "name" : rc['n'],
                            "number" : r["number"],
                            "direction": r["direction"],
                            "type": r["type"]
                            },
                        "geometry":{
                            "type": "Point",
                            "coordinates":[rc["lng"], rc["lat"]]
                            }
                        }
            stops_geojson['features'].append(feature)
                        

In [33]:
with open("stops2.geojson", 'w') as output:
    json.dump(stops_geojson, output)

In [30]:
stops_geojson['features'][0]

{'geometry': {'coordinates': [30.235457, 59.954937], 'type': 'Point'},
 'properties': {'direction': 'f',
  'name': 'Уральская улица',
  'number': '1',
  'rout': '/tram/1',
  'type': 'tram'},
 'type': 'Feature'}